## Import libraries

In [ ]:
import ee
import geemap
import os, re, json, time
import geopandas as gpd
from datetime import datetime, timedelta, date
from shapely.geometry import Polygon, mapping
from timezonefinder import TimezoneFinder
import pytz
from zoneinfo import ZoneInfo

## Start Connection with Google Earth Engine

In [ ]:
ee.Authenticate(force=True)
ee.Initialize(project = "project-ID") # specify the project name in the argument.
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## Set up environmental variables

In [ ]:
# To get root path under project folder structure
cwd = os.path.dirname(os.path.abspath("__file__"))
root_folder = cwd.split('\\')
root_folder = root_folder[0:-3]
if root_folder[-1] != 'fireRunSeverity':
    print("!!-- Didn't get correct root folder! --!!")
    print(root_folder)
    print("!!-- Modify rooting index at line: 18 and come back --!!")
if re.search(r":", root_folder[0]):
    root_folder = os.path.join(root_folder[0], os.sep, *root_folder[1:])
else:
    root_folder = os.path.join(os.sep,*root_folder)

# Local Side data
# List in put data for GEE fetch
if os.path.isfile('geeIDS.json'):
    with open('geeIDS.json', 'r', encoding='utf-8') as f:
        exist_indicesList_json = json.load(f)
run_DataDir = r"data/fireruns"
AreaList = os.listdir(os.path.join(root_folder,run_DataDir))
in_Name = AreaList[5]
in_Name

In [ ]:
# Check if any index map has been calculated
outGEEFLD = os.path.join(root_folder, 'data', 'GEE', in_Name)
if not os.path.isdir(outGEEFLD):
    os.makedirs(outGEEFLD)
exist_indicesList = os.listdir(outGEEFLD)
exist_indicesList = [f.split("--")[0] for f in exist_indicesList if os.path.isfile(os.path.join(outGEEFLD, f))]
if 'exist_indicesList_json' in locals():
    if in_Name in exist_indicesList_json.keys():
        for exist_id in exist_indicesList_json[in_Name]:
            if exist_id not in exist_indicesList:
                exist_indicesList.append(exist_id)
exist_indicesList

## Load EE side data

In [ ]:
# Load Online Data Collection (Run for all process)
# Add Earth Engine dataset
S2_harmon = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
L7_T1L2   = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
SRTM      = ee.Image("USGS/SRTMGL1_003")
SPEI      = ee.ImageCollection("CSIC/SPEI/2_10")


## Load Local side data

In [ ]:
print("\n\n ######################################################")
print("Process AOI:", in_Name)
dataFLD = os.path.join(root_folder, run_DataDir, in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        print(f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("No. of shp file not equal to 1!")
    exit()
else:
    shpIn = shpIn[0]

# Read Shp file
shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))
print(shpGPD.crs)
shp_reproj = shpGPD.to_crs("EPSG:4326")
# Get the bounds
minx, miny, maxx, maxy = shp_reproj.total_bounds
# Create a polygon of bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
# Convert the polygon to GeoJSON
json_polygon = mapping(bounds_polygon)
print(json_polygon)

## Modify time zone

In [ ]:
latitude = (miny + maxy)/2
longitude = (minx + maxx)/2
tf = TimezoneFinder()
timezone = tf.timezone_at(lng=longitude, lat=latitude)
print(f"Accurate Time Zone: {timezone}")

In [ ]:
# Get date of data
# Convert time zone to UTC
fire_dtString = list(shpGPD['FeHo'].dropna())
fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M").replace(tzinfo=ZoneInfo(timezone)) for dt in fire_dtString if not re.search("\\D", dt[:4])] # Filter out string with non-digit characters
tz_utc = pytz.timezone("UTC")
fire_dtsList_UTC = [dt.astimezone(tz_utc) for dt in fire_dtsList]

# print(sorted(fire_dtsList_UTC))
dateSt = min(fire_dtsList_UTC).date()
dateEd = max(fire_dtsList_UTC).date()
print("Fire start date and end date:")
print(dateSt, dateEd)

L7_flag = False # for using Landsat-7 dataset if burning timing before Sentinel-2
if dateSt < date(2017, 3, 28):
    print("!! ---------- Date before 28 Mar 2017! ---------- !!")
    print("----- AOI: ", in_Name, " -----")
    print("----- Using Landsat 7 ......")
    L7_flag = True

SPEI_flag = True
if dateSt > date(2023, 12, 31):
    print("----- Date after 2023/12 is not available -----")
    print("----- AOI: ", in_Name, " -----")
    SPEI_flag = False

# Compute the date period for retrieving Satellite img
prefire_date  = [dateSt - timedelta(days = 4*30), dateSt - timedelta(days = 1)]
postfire_date = [dateEd + timedelta(days = 1),    dateEd + timedelta(days = 4*30)]

prefire_date = [d.strftime("%Y-%m-%d") for d in prefire_date]
postfire_date = [d.strftime("%Y-%m-%d") for d in postfire_date]

print("Pre- and Post- fire periods selected for satellite images:")
print(prefire_date, postfire_date)

## Upload data to EE

In [ ]:
clipAOI = ee.Geometry.Polygon(json_polygon["coordinates"])
clipAOI2 = clipAOI.buffer(50000)
print(clipAOI.getInfo())

## (GEE) Process satellite

In [ ]:
# Functions for Computing Necessary indices
def func_maskClouds(image):
    imgCLP = image.select("MSK_CLDPRB")
    mskCLP = imgCLP.lt(10)  
    qa = image.select("QA60")
    Clouds = qa.bitwiseAnd(1 << 10).eq(0)  
    Cirrus = qa.bitwiseAnd(1 << 11).eq(0) 
    mask = Clouds.And(Cirrus).And(mskCLP)
    return image.updateMask(mask)

def maskClouds_L7(image):
    qa = image.select('QA_PIXEL')
    dilated_cloud = qa.bitwiseAnd(1 << 1).eq(0)
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    cloud_sha = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(cloud_sha).And(dilated_cloud)
    return image.updateMask(mask)

def func_rescale(image, scale, offset):
    refl = image.multiply(scale).add(offset)
    return refl.copyProperties(image, ["system:time_start", 'system:index'])

def func_calcIndices(image, RED, NIR, SWIR):
    NBR = image.normalizedDifference([NIR, SWIR]).rename("NBR") 
    NDVI = image.normalizedDifference([NIR, RED]).rename("NDVI") 
    return image.addBands([NBR, NDVI])

def func_calcIndices2(image):
    Cl_RBR   = image.expression(
        "dNBR / (preNBR + 1.001)", {
            "preNBR": image.select('Cl_preNBR'),
            "dNBR": image.select('Cl_dNBR') 
        }
    ).rename('Cl_RBR').toFloat()
    Cl_RdNBR = image.expression(
        "(abs(preNBR) < 0.001) ? dNBR/sqrt(0.001) : dNBR/sqrt(abs(preNBR))",{
            "preNBR": image.select('Cl_preNBR'),
            "dNBR": image.select('Cl_dNBR')
        }
    ).rename("Cl_RdNBR").toFloat()
    M_RBR   = image.expression(
        "dNBR / (preNBR + 1.001)", {
            "preNBR": image.select('M_preNBR'),
            "dNBR": image.select('M_dNBR') 
        }
    ).rename('M_RBR').toFloat()
    M_RdNBR = image.expression(
        "(abs(preNBR) < 0.001) ? dNBR/sqrt(0.001) : dNBR/sqrt(abs(preNBR))",{
            "preNBR": image.select('M_preNBR'),
            "dNBR": image.select('M_dNBR')
        }
    ).rename("M_RdNBR").toFloat()
    return image.addBands([Cl_RBR, Cl_RdNBR, M_RBR, M_RdNBR])

In [ ]:
# Filter data region and date
bandList = ["SR_B.", "SR_CLOUD_QA", "QA_PIXEL"] if L7_flag else ["B.", "B..", "QA60", "MSK_CLDPRB"]
band_SPEI = ["SPEI_03_month", "SPEI_06_month", "SPEI_09_month", "SPEI_12_month"]

if not L7_flag:
    S2_pre_select = S2_harmon.filterDate(prefire_date[0], prefire_date[1]) \
        .filterBounds(clipAOI2).select(bandList)
    S2_pos_select = S2_harmon.filterDate(postfire_date[0], postfire_date[1]) \
        .filterBounds(clipAOI2).select(bandList)
else:
    L7_pre_select = L7_T1L2.filterDate(prefire_date[0], prefire_date[1]) \
        .filterBounds(clipAOI2).select(bandList)
    L7_pos_select = L7_T1L2.filterDate(postfire_date[0], postfire_date[1]) \
        .filterBounds(clipAOI2).select(bandList)

In [ ]:
# Time compositing imageCollections
if not L7_flag:
    # Mosaicking composite
    S2pre = S2_pre_select.map(func_maskClouds) \
        .map(lambda image: func_rescale(image, scale=0.0001, offset=0)) \
        .sort('system:time_start') \
        .mosaic(); 
    S2pos = S2_pos_select.map(func_maskClouds) \
        .map(lambda image: func_rescale(image, scale=0.0001, offset=0)) \
        .sort('system:time_start', False) \
        .mosaic(); 
    # print(json.dumps(S2pre.getInfo(), indent=4))

    # Mean composite
    S2pre_mean = S2_pre_select.map(func_maskClouds) \
        .map(lambda image: func_rescale(image, scale=0.0001, offset=0)) \
        .mean(); 
    S2pos_mean = S2_pos_select.map(func_maskClouds) \
        .map(lambda image: func_rescale(image, scale=0.0001, offset=0)) \
        .mean(); 
    # print(json.dumps(S2pre_mean.getInfo(), indent=4))
else:  
    # Mosaicking composite
    L7pre = L7_pre_select.map(maskClouds_L7) \
        .map(lambda image: func_rescale(image, 0.0000275, -0.2)) \
        .sort('system:time_start')  \
        .mosaic()
    L7pos = L7_pos_select.map(maskClouds_L7) \
        .map(lambda image: func_rescale(image, 0.0000275, -0.2)) \
        .sort('system:time_start', False) \
        .mosaic()
    # print(json.dumps(L7pre.getInfo(), indent=4))
    
    # Mean composite
    L7pre_mean = L7_pre_select.map(maskClouds_L7) \
        .map(lambda image: func_rescale(image, 0.0000275, -0.2)) \
        .mean()
    L7pos_mean = L7_pos_select.map(maskClouds_L7) \
        .map(lambda image: func_rescale(image, 0.0000275, -0.2)) \
        .mean()
    # print(json.dumps(L7pre_mean.getInfo(), indent=4))


In [ ]:
if not L7_flag:
    # Indices Calculation
    pre_id   = func_calcIndices(S2pre,      RED="B4", NIR="B8", SWIR="B12")
    pos_id   = func_calcIndices(S2pos,      RED="B4", NIR="B8", SWIR="B12")
    pre_id_m = func_calcIndices(S2pre_mean, RED="B4", NIR="B8", SWIR="B12")
    pos_id_m = func_calcIndices(S2pos_mean, RED="B4", NIR="B8", SWIR="B12")
else:  
    # Indices Calculation
    pre_id   = func_calcIndices(L7pre,      RED="SR_B3", NIR="SR_B4", SWIR="SR_B7")
    pos_id   = func_calcIndices(L7pos,      RED="SR_B3", NIR="SR_B4", SWIR="SR_B7")
    pre_id_m = func_calcIndices(L7pre_mean, RED="SR_B3", NIR="SR_B4", SWIR="SR_B7")
    pos_id_m = func_calcIndices(L7pos_mean, RED="SR_B3", NIR="SR_B4", SWIR="SR_B7")


# dNBR
indices_first  = pre_id.select("NBR").subtract(pos_id.select("NBR")).multiply(1000).rename("Cl_dNBR")
indices_first  = indices_first.addBands(
    pre_id_m.select("NBR").subtract(pos_id_m.select("NBR")).multiply(1000).rename("M_dNBR")
)

# Other indices
indices_first  = indices_first.addBands([pre_id.select("NBR").rename("Cl_preNBR"), 
                                            pos_id.select("NBR").rename("Cl_posNBR"),
                                            pre_id_m.select("NBR").rename("M_preNBR"), 
                                            pos_id_m.select("NBR").rename("M_posNBR"),])

indices_final = func_calcIndices2(indices_first)

# Environmental factors
terrain = ee.Terrain.products(SRTM.clip(clipAOI2));
indices_final = indices_final.addBands([SRTM.clip(clipAOI2).select('elevation').rename('env_elevation'),
                                        terrain.select('aspect').rename('env_aspect'),])

# SPEI indices
if SPEI_flag:
    month_SPEI = (date(dateSt.year, dateSt.month, 1).strftime("%Y-%m-%d"), date(dateSt.year, dateSt.month, 28).strftime("%Y-%m-%d"))
    spei_flt = SPEI.filterDate(month_SPEI[0], month_SPEI[1]) \
        .filterBounds(clipAOI2).first() # Select the only image
    indices_final = indices_final.addBands(spei_flt.select(band_SPEI))

indices_final

## Output image to Google drive folder

In [ ]:
# Select needed bands
bd = indices_final.bandNames()
bd = bd.removeAll(['Cl_preNBR', 'Cl_posNBR', 'M_preNBR', 'M_posNBR'])
indices_for_output = indices_final.select(bd)
bd = indices_for_output.bandNames().getInfo()
print(bd)

In [ ]:
# Define export parameters
print("#-----------------#")
print("Exporting for", in_Name)
driveFLD = os.path.join('EarthEngineFolder', in_Name, in_Name)
tStamp = datetime.now().strftime("%d%m%Y_%H%M")
datasetName = "L7" if L7_flag else "S2"
for bd_i in bd:
    if f'{datasetName}_{bd_i}' in exist_indicesList:
        # skip map production if index maps already exist in local folder
        continue

    export_task = ee.batch.Export.image.toDrive(
        image=indices_for_output.select(bd_i),
        description='sentinel2_fire_indices_image_export',
        folder='EarthEngineFolder',  # Folder in your Google Drive
        fileNamePrefix=f'PythonGEE_output--{in_Name}--{datasetName}_{bd_i}--{tStamp}',  # File name prefix
        region=clipAOI,  # Define the region to export (could be an area of interest)
        scale=10,  # Spatial resolution in meters
        fileFormat='GeoTIFF',  # File format
        maxPixels=1e12  # Max pixels to export
    )

    # Start the export task
    try:
        export_task.start()
        print("Band: ", bd_i)
        print("Exporting to Google Drive...")

        # Check the status of the export task
        print("Exporting... Please wait")
        while export_task.active():
            print('.',  end="")
            time.sleep(10)

        print('Export completed.')

    except ee.ee_exception.EEException as e:
        print(f"Error during export: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    print("----")

## Miscellaneous

#### Miscellaneous: Counting pixels

In [ ]:
# pixel_count = S2_dNBR.select('dNBR').reduceRegion(
#     reducer=ee.Reducer.count(),
#     geometry=clipAOI,  # Use the image's geometry as the region
#     scale=10,  # Spatial resolution in meters
#     maxPixels=1e10  # Maximum number of pixels to process
# )

# # Print the pixel count
# print('Pixel count for B4 band:', pixel_count.getInfo())